In [1]:
%matplotlib inline 

from fnmatch import fnmatch

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import requests
from pattern import web

# set some nicer defaults for matplotlib
from matplotlib import rcParams
positions=('QB','RB','RB','WR','WR','TE','FLEX','D','K','Total')
leagueid=732290
year=2016
weeks=4

def team_list():
    url="http://games.espn.com/ffl/leagueoffice?leagueId="+str(leagueid)+"&seasonId="+str(year)
    xml=requests.get(url).text
    dom = web.Element(xml).by_tag('li')
    lis=[]

    i=0
    while len(str(dom[i+66]))>80:
        temp=dom[i+66]
        temp = str(temp)
        lis.append(temp[130:len(temp)-46])
        i=i+1
    return lis


In [2]:
numteams=len(team_list())

def get_points(t,w):
    points=[0,0,0,0,0,0,0,0,0,0]
    maxpoints=[0,0,0,0,0,0,0,0,0,0]
    url="http://games.espn.com/ffl/boxscorequick?leagueId="+str(leagueid)+"&teamId="+str(t)+"&scoringPeriodId="+str(w)+"&seasonId="+str(year)+"&view=scoringperiod&version=quick"
    xml=requests.get(url).text
    dom = web.Element(xml).by_tag('tr')

    for i in range(9):
        temp=dom[4+i] #4 though 12 is QB to K, 15 to 20 is Bench
        if str(temp).find('playerid')<0:
            temp=dom[6+i]
            
        if len(temp.by_tag('td'))==4:
            temp2=str(temp.by_tag('td')[3])
        else:
            temp2=str(temp.by_tag('td')[4]) #gets to the points section
        if temp2.find('--')>0:
            temp2=0
        else:
            scorestart=temp2.find('</td>')-14+temp2[(temp2.find('</td>')-15):temp2.find('</td>')].find('>')
            scoreend=temp2.find('</td>')-15+temp2[(temp2.find('</td>')-15):temp2.find('</td>')+1].find('<')
            temp2=temp2[scorestart:scoreend] #takes out the points
        points[i]=float(temp2)
        points[9]=float(temp2)+points[9]
      
    maxpoints=points*1
    for i in range(6):
        tempscores=[0,0,0]
        benchscore=0
        temp=dom[15+i] #4 though 12 is QB to K, 15 to 20 is Bench
        if str(temp).find('playerid')<0:
            temp=dom[17+i]
            
        if len(temp.by_tag('td'))==4:
            temp2=str(temp.by_tag('td')[3])
        else:
            temp2=str(temp.by_tag('td')[4]) #gets to the points section
        if temp2.find('--')>0:
            temp2=0
        else:
            scorestart=temp2.find('</td>')-14+temp2[(temp2.find('</td>')-15):temp2.find('</td>')].find('>')
            scoreend=temp2.find('</td>')-15+temp2[(temp2.find('</td>')-15):temp2.find('</td>')+1].find('<')
            temp2=temp2[scorestart:scoreend] #takes out the points
            benchscore=float(temp2)
        
        temp=str(temp)
        if temp.find('QB')>0:
            if maxpoints[0]<benchscore:
                maxpoints[0]=benchscore
        if temp.find('RB')>0:
            tempscores=[maxpoints[1],maxpoints[2],benchscore]
            tempscores=sorted(tempscores,reverse=True)
            maxpoints[1]=tempscores[0]
            maxpoints[2]=tempscores[1]
            if maxpoints[6]<tempscores[2]:
                maxpoints[6]=tempscores[2]
        if temp.find('WR')>0:
            tempscores=[maxpoints[3],maxpoints[4],benchscore]
            tempscores=sorted(tempscores,reverse=True)
            maxpoints[3]=tempscores[0]
            maxpoints[4]=tempscores[1]
            if maxpoints[6]<tempscores[2]:
                maxpoints[6]=tempscores[2]
        if temp.find('TE')>0:
            tempscores=[maxpoints[5],benchscore,0]
            tempscores=sorted(tempscores,reverse=True)
            maxpoints[5]=tempscores[0]
            if maxpoints[6]<tempscores[1]:
                maxpoints[6]=tempscores[1]
        if temp.find('D/ST')>0:
            if maxpoints[7]<benchscore:
                maxpoints[7]=benchscore
        if temp.find('nbsp;K')>0:
            if maxpoints[7]<benchscore:
                maxpoints[7]=benchscore
    maxpoints[9]=0
    for i in range(9):
        maxpoints[9]=maxpoints[9]+maxpoints[i]                  
    return points, maxpoints

print get_points(7,1)

([23.6, 6.0, 9.8, 12.2, 6.8, 1.1, 2.3, 5.0, 9.0, 75.80000000000001], [26.1, 11.5, 9.8, 12.2, 6.8, 13.1, 6.0, 5.0, 9.0, 99.5])


In [3]:
#MAKES EXCELL FILE OF JUST TOTAL POINTS
teams=team_list()
with open('HopkinsFF_maxpoints.csv','w') as out_file:
    Header='Team'
    for week in range(weeks):
        Header+=',Week'+str(week+1)
    out_file.write(Header+'\n')
    for i in range(len(teams)):
        out_string=str(teams[i])
        if i<5:
            t=i+1
        elif i<11:
            t=i+2
        else:
            t=14
        #t=i+1
        for week in range(weeks):
            temp=get_points(t,week+1)[0]
            out_string+=","+str(temp[9])
        out_string+="\n"
        for week in range(weeks):
            temp=get_points(t,week+1)[1]
            out_string+=","+str(temp[9])    
        out_string+="\n"
        out_file.write(out_string)